In [1]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

# Define dataset directory
dataset_dir = 'craft_it_dataset'

# Check the contents of each folder
categories = ['can', 'cardboard', 'paper', 'plastic', 'plastic botle']

for category in categories:
    path = os.path.join(dataset_dir, category)
    print(f"{category} has {len(os.listdir(path))} images")


can has 1060 images
cardboard has 891 images
paper has 1050 images
plastic has 865 images
plastic botle has 1508 images


In [3]:
from sklearn.model_selection import train_test_split

# Define image size and batch size
IMG_SIZE = (150, 150)
BATCH_SIZE = 32

# Create a function to split data into training and testing sets
def create_data_generators(dataset_dir, img_size, batch_size):
    data_gen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
    
    train_gen = data_gen.flow_from_directory(
        dataset_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='training'
    )
    
    val_gen = data_gen.flow_from_directory(
        dataset_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation'
    )
    
    return train_gen, val_gen

# Create data generators
train_gen, val_gen = create_data_generators(dataset_dir, IMG_SIZE, BATCH_SIZE)


Found 4300 images belonging to 5 classes.
Found 1074 images belonging to 5 classes.


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(5, activation='softmax')  # 5 classes: can, cardboard, paper, plastic, plastic botle
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


C:\Users\bacht\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │    18,940,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │         2,565 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,036,229 (72.62 MB)

 Trainable params: 19,036,229 (72.62 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
# Train the model
history = model.fit(
    train_gen,
    epochs=25,
    validation_data=val_gen
)


Epoch 1/25


C:\Users\bacht\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


135/135 ━━━━━━━━━━━━━━━━━━━━ 206s 1s/step - accuracy: 0.4170 - loss: 1.5629 - val_accuracy: 0.5922 - val_loss: 0.9394
Epoch 2/25
135/135 ━━━━━━━━━━━━━━━━━━━━ 105s 764ms/step - accuracy: 0.7253 - loss: 0.7396 - val_accuracy: 0.6555 - val_loss: 0.9694
Epoch 3/25
135/135 ━━━━━━━━━━━━━━━━━━━━ 64s 471ms/step - accuracy: 0.7757 - loss: 0.5986 - val_accuracy: 0.6993 - val_loss: 0.8085
Epoch 4/25
135/135 ━━━━━━━━━━━━━━━━━━━━ 100s 736ms/step - accuracy: 0.8442 - loss: 0.4251 - val_accuracy: 0.6881 - val_loss: 0.9330
Epoch 5/25
135/135 ━━━━━━━━━━━━━━━━━━━━ 168s 1s/step - accuracy: 0.8488 - loss: 0.3848 - val_accuracy: 0.7132 - val_loss: 0.8088
Epoch 6/25
135/135 ━━━━━━━━━━━━━━━━━━━━ 67s 494ms/step - accuracy: 0.8897 - loss: 0.3103 - val_accuracy: 0.6536 - val_loss: 1.0146
Epoch 7/25
135/135 ━━━━━━━━━━━━━━━━━━━━ 65s 480ms/step - accuracy: 0.8825 - loss: 0.3075 - val_accuracy: 0.7142 - val_loss: 0.9137
Epoch 8/25
135/135 ━━━━━━━━━━━━━━━━━━━━ 61s 449ms/step - accuracy: 0.9227 - loss: 0.2122 - val_a

In [ ]:
# Plot training & validation accuracy and loss
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.figure(figsize=(12, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()


In [ ]:
# Save the trained model
model.save('recycling_classifier_cnn_model.h5')
